# Academy Awards Part II
This is a continuation of the previous Dataquest project on SQL. Now that the data is written in an SQL database, we can normalise it.

In [2]:
import sqlite3 as sql

conn = sql.connect("nominations.db")

query_one = "pragma table_info(nominations)"
schema = conn.execute(query_one).fetchall()
query_two = "SELECT * FROM nominations LIMIT 10"
first_ten = conn.execute(query_two).fetchall()
first_ten

for s in schema:
    print(s)
for i in first_ten:
    print(i)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)
(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting 

## Ceremonies Table

In [6]:
create_ceremonies = "CREATE TABLE ceremonies(id integer PRIMARY KEY,Year integer,Host text);"
conn.execute(create_ceremonies)

In [7]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"
conn.executemany(insert_query, years_hosts)

query = "SELECT * FROM ceremonies LIMIT 10"
conn.execute(query).fetchall()


[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [8]:
conn.execute("PRAGMA foreign_keys = ON")

## Ceremony to Nominations Table One-to-Many Relationship

In [11]:
create_nom_two ="CREATE TABLE nominations_two(id integer PRIMARY KEY,category text,nominee text,movie text,character text,won integer,ceremony_id integer,FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id))"
conn.execute(create_nom_two)

In [13]:
join_query = "SELECT nominations.category,nominations.nominee,nominations.movie,nominations.character,nominations.won,ceremonies.id FROM nominations INNER JOIN ceremonies ON nominations.year == ceremonies.year"
joined_nominations = conn.execute(join_query).fetchall()

In [15]:
insert_noms_two = "INSERT INTO nominations_two (category,nominee,movie,character,won,ceremony_id) VALUES (?,?,?,?,?,?)"
conn.executemany(insert_noms_two, joined_nominations)
print(conn.execute("SELECT * FROM nominations_two LIMIT 5").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1), (2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1), (3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1), (4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1), (5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)]


In [16]:
conn.execute("DROP TABLE nominations")
conn.execute("ALTER TABLE nominations_two RENAME TO nominations")

## Movie, Actor, and Movies and Actors Join Table for Many-to-Many Relationship

In [17]:
create_movie = "CREATE TABLE movies(id integer PRIMARY KEY,movie text)"
conn.execute(create_movie)

In [18]:
create_actors = "CREATE TABLE actors(id integer PRIMARY KEY,actor text)"
conn.execute(create_actors)

In [19]:
create_movies_actors = "CREATE TABLE movies_actors(id integer PRIMARY KEY,movie_id integer REFERENCES movies(id),actor_id integer REFERENCES actors(id))"
conn.execute(create_movies_actors)

In [23]:
insert_movies = "INSERT INTO movies (movie) SELECT DISTINCT movie FROM nominations"
insert_actors = "INSERT INTO actors (actor) SELECT DISTINCT nominee FROM nominations"
conn.execute(insert_movies)
conn.execute(insert_actors)

print(conn.execute("SELECT * FROM movies LIMIT 5").fetchall())
print(conn.execute("SELECT * FROM actors LIMIT 5").fetchall())


[(1, 'Biutiful'), (2, 'True Grit'), (3, 'The Social Network'), (4, "The King's Speech"), (5, '127 Hours')]
[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]


In [35]:
join_query = "SELECT movie,nominee FROM nominations"
movie_actors_list = conn.execute(join_query).fetchall()

movies_actors_insert = "INSERT INTO movies_actors(movie_id,actor_id) VALUES ((SELECT id FROM movies WHERE movie == ?),(SELECT id FROM actors WHERE actor == ?))"
conn.executemany(movies_actors_insert,movie_actors_list)

conn.execute("SELECT * FROM movies_actors LIMIT 10").fetchall()


[(1, 1, 1),
 (2, 2, 2),
 (3, 3, 3),
 (4, 4, 4),
 (5, 5, 5),
 (6, 6, 6),
 (7, 7, 7),
 (8, 8, 8),
 (9, 9, 9),
 (10, 4, 10)]